In [ ]:
############ LOAD in custom packages ################
import sys
import os
import pandas as pd
import matplotlib.pyplot as plt
import importlib
import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt

from sklearn.pipeline import Pipeline
from sklearn.preprocessing import OneHotEncoder, OrdinalEncoder, StandardScaler
from sklearn.impute import SimpleImputer, KNNImputer
from sklearn.base import BaseEstimator, TransformerMixin
from sklearn.compose import ColumnTransformer
from sklearn.model_selection import train_test_split, cross_val_score, RandomizedSearchCV, GridSearchCV, KFold
from sklearn.linear_model import LogisticRegression, LinearRegression, SGDRegressor
from sklearn.ensemble import RandomForestClassifier, AdaBoostClassifier, GradientBoostingClassifier, BaggingClassifier, RandomForestRegressor
from sklearn.neighbors import KNeighborsClassifier, KNeighborsRegressor
from sklearn.svm import LinearSVC, SVR
from sklearn.tree import DecisionTreeClassifier, DecisionTreeRegressor
from sklearn.metrics import ConfusionMatrixDisplay, f1_score, make_scorer, confusion_matrix, mean_squared_error, mean_absolute_error

from statsmodels.stats.outliers_influence import variance_inflation_factor
from statsmodels.tools.tools import add_constant

from xgboost import XGBClassifier, XGBRegressor
from catboost import CatBoostClassifier

from scipy.stats import randint, uniform


# Get the absolute path of the project root
project_root = os.path.abspath(os.path.join(os.getcwd(), ".."))
# Define data directory
brighten_dir = os.path.join(project_root, 'BRIGHTEN_data')

# Add project root to sys.path
sys.path.append(project_root)

# Import and reload custom scripts
from scripts import preprocessing as pre
from scripts import visualization as vis
from scripts import feature_selection as fs
from scripts import clustering as cl
importlib.reload(pre)
importlib.reload(vis)
importlib.reload(fs)
importlib.reload(cl)

################ DEFINE column variables from data ###################
from scripts.variables import id_columns, daily_cols_v1, daily_v2_common 
from scripts.variables import phq2_cols, phq9_cols, weekly_cols, passive_cols, survey_cols


## Load in dfs scaled
df_names = ['v1_day', 'v2_day', 'v1_week', 'v2_week']

dfs_pca = {}
for name in df_names:
    dfs_pca[name] = pd.read_csv(os.path.join(brighten_dir, f'{name}_pca.csv'))

dfs_scaled = {}
for name in df_names:
    dfs_scaled[name] = pd.read_csv(os.path.join(brighten_dir, f'{name}_scaled.csv'))


In [2]:
#### Simple linear regression
from sklearn.linear_model import LinearRegression
# Step 1: Set x_cols to the various V1 PCs
y_var = 'pc_depression_phq2'
data = pd.read_csv(os.path.join(brighten_dir, 'v1_day_pca.csv'))
print(data.columns.to_list())
results = pd.DataFrame()
count=0
x_cols = [col for col in data.columns.to_list() if 'pc_' in col and col!=y_var]
y_col = [y_var]

reg = LinearRegression().fit(data[x_cols], data[y_col])
print(reg.score(data[x_cols], data[y_col]))
print(reg.coef_)
print(reg.intercept_)

['participant_id', 'num_id', 'dt', 'week', 'day', 'gender', 'age', 'aggregate_communication', 'call_count', 'call_duration', 'interaction_diversity', 'missed_interactions', 'mobility', 'mobility_radius', 'sms_count', 'sms_length', 'unreturned_calls', 'phq2_1', 'phq2_2', 'phq2_sum', 'pc_communication', 'pc_missed_communications', 'pc_mobility', 'pc_depression_phq2']
0.012852891598219895
[[0.05172084 0.09099842 0.08534029]]
[-0.00746926]


In [3]:
######### Mixed LM Model V1 vs phq2_sum #########
###### V1 PCs vs phq2_sum #########
from statsmodels.regression.mixed_linear_model import MixedLM
import statsmodels.api as sm
import statsmodels.formula.api as smf

ignore_cols = ['num_id', 'dt', 'week', 'day']
y_var = 'phq9_sum'
data = pd.read_csv(os.path.join(brighten_dir, 'v1_week.csv'))
results = pd.DataFrame()
count = 0
x_cols = [col for col in data.columns.to_list() if 'phq9' not in col and col not in ignore_cols]
data_full = data[x_cols + [y_var, 'num_id']].copy().dropna()
data_full["num_id"] = data_full["num_id"].astype("category")
display(data_full)
from statsmodels.stats.outliers_influence import variance_inflation_factor
import pandas as pd

model = smf.mixedlm(f"{y_var} ~ mobility  + sms_count + unreturned_calls + phq2_sum", 
                    data=data_full, 
                    groups=data_full["num_id"])
result = model.fit()
print(f'Relationship to y_col {y_var}, Nested Linear Regression on "num_id"')
print(result.summary())



,participant_id,aggregate_communication,call_count,call_duration,interaction_diversity,missed_interactions,mobility,mobility_radius,sms_count,sms_length,unreturned_calls,phq2_1,phq2_2,phq2_sum,phq9_sum,num_id
1184,BLUE-00062,38.0,7.0,5412.0,7.0,1.0,9.334,5.597,31.0,3336.0,0.0,3.000000,2.000000,5.000000,9.0,21
1260,BLUE-00063,26.0,3.0,158.0,2.0,0.0,3.123,33.944,23.0,978.0,0.0,2.000000,1.000000,3.000000,10.0,22
1574,BLUE-00066,123.0,10.0,9902.0,8.0,5.0,1.536,3.223,113.0,6698.0,2.0,2.000000,2.000000,4.000000,8.0,25
1588,BLUE-00066,118.0,10.0,5551.0,10.0,6.0,0.990,11.427,108.0,5457.0,6.0,2.000000,2.000000,4.000000,8.0,25
1602,BLUE-00066,104.0,7.0,772.0,7.0,2.0,0.338,0.017,97.0,5701.0,0.0,2.000000,2.000000,4.000000,12.0,25
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
79687,YELLOW-00260,9.0,2.0,126.0,2.0,0.0,0.444,11.349,7.0,355.0,0.0,1.000000,1.000000,2.000000,3.0,2071
79758,YELLOW-00262,259.0,3.0,4563.0,8.0,1.0,2.320,3.731,256.0,17701.0,0.0,2.833333,2.833333,5.666667,6.0,2073
79765,YELLOW-00262,130.0,3.0,3557.0,11.0,8.0,1.177,2.243,127.0,6712.0,8.0,2.000000,3.000000,5.000000,10.0,2073
79782,YELLOW-00263,61.0,26.0,14992.0,16.0,1.0,0.845,0.124,35.0,951.0,0.0,5.000000,5.000000,10.000000,12.0,2074


Relationship to y_col phq9_sum, Nested Linear Regression on "num_id"
           Mixed Linear Model Regression Results
Model:             MixedLM  Dependent Variable:  phq9_sum  
No. Observations:  784      Method:              REML      
No. Groups:        223      Scale:               8.8331    
Min. group size:   1        Log-Likelihood:      -2152.5755
Max. group size:   8        Converged:           Yes       
Mean group size:   3.5                                     
-----------------------------------------------------------
                 Coef.  Std.Err.   z    P>|z| [0.025 0.975]
-----------------------------------------------------------
Intercept         4.007    0.465  8.623 0.000  3.096  4.917
mobility          0.031    0.100  0.312 0.755 -0.165  0.228
sms_count        -0.000    0.003 -0.122 0.903 -0.005  0.005
unreturned_calls  0.092    0.087  1.054 0.292 -0.079  0.264
phq2_sum          1.052    0.071 14.711 0.000  0.912  1.192
Group Var        11.637    0.609          

5. Model Selection - Classfication

We'll start by using several of the most popular classifiers with default parameters and compare how well they perform to indentify which might be best to proceed with.


In [ ]:


# Define the classifiers to test
clfs = [
    ('Logistic Regression', LogisticRegression(solver='liblinear', max_iter=2000)),
    ('KNN', KNeighborsClassifier()),
    ('Decision Tree', DecisionTreeClassifier()),
    ('Random Forest', RandomForestClassifier(random_state=42)),
    ('Linear SVM', LinearSVC(random_state=42, max_iter=1000, dual='auto')),
    ('XGBoost', XGBClassifier(random_state=42)),
    ('AdaBoost', AdaBoostClassifier(random_state=42, algorithm='SAMME')),
    ('Gradient Boost', GradientBoostingClassifier(random_state=42)),
    ('Bagging', BaggingClassifier(random_state=42)),
    ('CatBoost', CatBoostClassifier(random_state=42, verbose=0)),
]

#We'll use cross-validation to get a better understanding of each models performance, rather than just a single test. Let's create a KFold object so we can use the same folds for each classifier.

# Create KFold object with 10 folds
kf = KFold(n_splits=10, shuffle=True, random_state=42)

# Dictionary to store results
results = {}

# Evaluate each classifier using cross-validation
for clf_name, clf in clfs:
    cv_scores = cross_val_score(clf, X_train_cleaned, y_train, cv=kf)
    results[clf_name] = cv_scores

cv_scores_df = pd.DataFrame(results)

#Let's now plot the results of each test, for each classifyer with a boxplot so we can compare the models.

# Plot scores
fig, ax = plt.subplots(figsize=(14, 8))
sns.boxplot(cv_scores_df)

# Add axis labels
ax.set_xlabel('Classifier', fontsize=12)
ax.set_ylabel('CV Accuracy Score', fontsize=12)
ax.set_title('Cross-Validation Scores for Different Classifiers', fontsize=14)

